In [ ]:
import sys
import shutil
sys.path.append('../dependencies/')
import pyemu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import pathlib as pl
from flopy.plot import styles as fpstyles


# Method of Morris First

In [ ]:
pst_name='./MASTER_SEN/global_sens_morris.pst'

In [ ]:
pst = pyemu.Pst(pst_name)

In [ ]:
morris_sum_file = pst_name.replace('.pst','.msn')

In [ ]:
morris_df = pd.read_csv(morris_sum_file, index_col=0).T.rename(columns={'global_s':'Storage','global_t':'Transmissivity',
                                                                       'well_1013__stream_apportionment121':'Existing Depletion'}).T
morris_df

In [ ]:
ax = morris_df.sort_values(by='sen_mean_abs',ascending=False).iloc[:25].sen_mean_abs.apply(np.log10).plot.bar()
plt.xlabel('Parameter Group')
plt.ylabel('Mean Absolute Sensitivity')


# Sobol' sensitivity

In [ ]:
pst_name='./MASTER_SEN/global_sens_sobol.pst'
si = pd.read_csv(pst_name.replace(".pst",".sobol.si.csv")).rename(columns={'output':'output','global_s':'Storage','global_t':'Transmissivity',
                                                                    'well_1013__stream_apportionment121':'Existing Depletion'})

In [ ]:
si

In [ ]:
sti = pd.read_csv(pst_name.replace(".pst",".sobol.sti.csv")).rename(columns={'output':'output','global_s':'Storage','global_t':'Transmissivity',
                                                                    'well_1013__stream_apportionment121':'Existing Depletion'})

In [ ]:
sti.iloc[1:].sum().iloc[1:].sort_values(ascending=False).astype(np.float64).apply(np.log10).plot.bar().grid(axis='y')

In [ ]:
si.iloc[1:].sum().iloc[1:].sort_values(ascending=False).astype(np.float64).apply(np.log10).plot.bar().grid(axis='y')

In [ ]:
si.iloc[1:].sum().iloc[1:].sort_values(ascending=False)

# Now make a combined plot for the paper

In [ ]:
with fpstyles.USGSPlot():
    fig,ax =plt.subplots(1,2,figsize=(6,4))
    morris_df.sort_values(by='sen_mean_abs',ascending=False).iloc[:25].sen_mean_abs.apply(np.log10).plot.bar(ax=ax[0]).grid(axis='y')
    ax[0].set_xlabel('Parameter Group')
    ax[0].set_ylabel('Mean Absolute Sensitivity')
    sti.iloc[1:].sum().iloc[1:].sort_values(ascending=False).astype(np.float64).apply(np.log10).plot.bar(ax=ax[1]).grid(axis='y')
    ax[1].set_xlabel('Parameter Group')
    ax[1].set_ylabel('Total Sensitivity')
    ax[0].set_title('Method of Morris')
    ax[1].set_title("Method of Sobol'")
    plt.suptitle("Global Sensitivity Analysis Results")
    plt.tight_layout()
    plt.savefig('./global_sens.pdf')


In [ ]:
ax